# I. Traitement de données en Python

## Contexte :

Dans le cadre de préparations de données de santé anonymisées pour la recherche, vous disposez de deux fichiers CSV compressés :

- patients.csv.gz : Liste de patients fictifs (patient_id, birth_date, gender).

- consultations.csv.zip : Historique de consultations (consultation_id, patient_id, date_consultation, diagnostic).

In [463]:
# Lib

from pathlib import Path
import pandas as pd
import numpy as np

### 1. Lecture des données

In [464]:
# Chargement des deux fichiers dans des Dataframes


# Détection automatique de la racine du projet
project_root = Path.cwd().parent   # notebook/ → projet
inputs_dir = project_root / "inputs"
outputs_dir = project_root / "outputs"

patients_path = inputs_dir / "patients.csv.gz"
consultations_path = inputs_dir / "consultations.csv.zip"

# Lecture du fichier patients (gzip)
df_patients = pd.read_csv(
    patients_path,
    compression="gzip",
    encoding="latin-1"
)

# Lecture du fichier consultations (zip)
df_consultations = pd.read_csv(
    consultations_path,
    compression="zip",
    encoding="latin-1"
)

# Vérification
print("Patients :", df_patients.shape)
print("Consultations :", df_consultations.shape)

Patients : (105, 3)
Consultations : (205, 4)


In [465]:
df_patients.head(10)

,patient_id,birth_date,gender
0,p_094,NaN,unknown
1,,NaN,unknown
2,NaN,1977-08-20,NaN
3,,1942-09-03,F
4,NaN,NaN,unknown
5,,NaN,NaN
6,p_069,not_a_date,NaN
7,NaN,N/ A,NaN
8,p_020,NaN,unknown
9,NaN,1970-03-13,NaN


In [466]:
df_consultations.head(10)

,consultation_id,patient_id,date_consultation,diagnostic
0,NaN,x_003,not_a_date,NaN
1,NaN,x_019,29/02/2020,débuté
2,c_054,x_046,NaN,à refaire
3,c_198,x_077,not_a_date,débuté
4,,x_018,not_a_date,NaN
5,NaN,p_040,NaN,nnull
6,c_063,p_099,25/02/2020,débuté
7,,x_069,not_a_date,nnull
8,NaN,p_097,NaN,nnull
9,NaN,p_060,NaN,terminé


### 2. Nettoyage & Typage

#### 2.1 Patients

In [467]:
# Gérer les valeurs nulles et/ou non-signifiantes.

from IPython.display import display

# Analyse du DF 'df_patients'

# Exploration rapide des données

print("\n=== Informations générales ===")
display(df_patients.info())

print("\n=== Statistiques descriptives (toutes colonnes) ===")
display(df_patients.describe(include="all"))

print("\n=== Valeurs uniques par colonne (hors NaN) ===")
for col in df_patients.columns:
    print(f"\n--- {col} ---")
    display(df_patients[col].dropna().unique())

print("\n=== Nombre de valeurs nulles par colonne ===")
display(df_patients.isna().sum())

print("=== Nombre de doublons par colonne ===")

for col in df_patients.columns:
    nb_doublons = df_patients[col].duplicated().sum()
    print(f"{col} : {nb_doublons}")


=== Informations générales ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   patient_id  59 non-null     object
 1   birth_date  65 non-null     object
 2   gender      43 non-null     object
dtypes: object(3)
memory usage: 2.6+ KB


None


=== Statistiques descriptives (toutes colonnes) ===


,patient_id,birth_date,gender
count,59,65,43
unique,30,23,3
top,,not_a_date,unknown
freq,30,27,24



=== Valeurs uniques par colonne (hors NaN) ===

--- patient_id ---


...


--- gender ---


array(['unknown', 'F', 'M'], dtype=object)


=== Nombre de valeurs nulles par colonne ===


patient_id    46
birth_date    40
gender        62
dtype: int64

=== Nombre de doublons par colonne ===
patient_id : 74
birth_date : 81
gender : 101


In [468]:
# La colonne 'patient_id' contient 59 valeurs non nulles, 46 valeurs nulles, 30 valeurs uniques et une valeur non signifiante qui est : ' '.
# La colonne 'birth_date' contient 65 valeurs non nulles, 40 valeurs nulles, 23 valeurs uniques et 2 valeurs non signifiantes qui sont : 'not_a_date' et 'N/ A'.
# la colonne 'gender' contient 43 valeurs non nulles, 62 valeurs nulles, 3 valeurs uniques et une valeur non signifiante : 'unknown'.


In [469]:
# vérification
n = len(df_patients)

check = {}
for col in df_patients.columns:
    dup = df_patients[col].duplicated().sum()
    nun = df_patients[col].nunique(dropna=False)
    check[col] = {"duplicates": dup, "nunique_including_NaN": nun, "n - nunique": n - nun}

import pandas as pd
display(pd.DataFrame(check).T)

,duplicates,nunique_including_NaN,n - nunique
patient_id,74,31,74
birth_date,81,24,81
gender,101,4,101


In [470]:
# Nettoyage des valeurs non-signifiantes

df_patients["patient_id"] = (
    df_patients["patient_id"]
    .str.strip()         # supprime les espaces
    .replace("", np.nan) # remplace les chaînes vides par NaN
)

df_patients["birth_date"] = df_patients["birth_date"].replace(
    ["not_a_date", "N/ A"], np.nan
)

df_patients["gender"] = df_patients["gender"].replace("unknown", np.nan)

In [471]:
# Vérification après nettoyage

print("\n=== Valeurs uniques par colonne (hors NaN) ===")
for col in df_patients.columns:
    print(f"\n--- {col} ---")
    display(df_patients[col].dropna().unique())

print("\n=== Nombre de valeurs nulles par colonne ===")
display(df_patients.isna().sum())


=== Valeurs uniques par colonne (hors NaN) ===

--- patient_id ---


array(['p_094', 'p_069', 'p_020', 'p_078', 'p_037', 'p_068', 'p_092',
       'p_065', 'p_023', 'p_018', 'p_075', 'p_039', 'p_091', 'p_038',
       'p_019', 'p_095', 'p_012', 'p_062', 'p_050', 'p_000', 'p_077',
       'p_064', 'p_036', 'p_061', 'p_047', 'p_011', 'p_071', 'p_072',
       'p_034'], dtype=object)


--- birth_date ---


array(['1977-08-20', '1942-09-03', '1970-03-13', '1981-03-20',
       '1972-09-23', '1942-11-07', '2008-06-30', '1996-05-25',
       '1998-02-19', '1973-12-16', '2000-09-16', '1948-09-04',
       '2009-05-09', '1987-12-02', '1959-10-11', '1940-07-31',
       '1947-10-19', '1997-05-13', '1949-03-03', '1964-12-15',
       '1980-09-04'], dtype=object)


--- gender ---


array(['F', 'M'], dtype=object)


=== Nombre de valeurs nulles par colonne ===


patient_id    76
birth_date    82
gender        86
dtype: int64

In [472]:
# Typage explicite des colonnes

# Identifiant patient : chaîne de caractères
df_patients["patient_id"] = df_patients["patient_id"].astype("string")
# Date de naissance : datetime
df_patients["birth_date"] = pd.to_datetime(
    df_patients["birth_date"]
)
# Genre : variable catégorielle
df_patients["gender"] = df_patients["gender"].astype("category")


In [473]:
df_patients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   patient_id  29 non-null     string        
 1   birth_date  23 non-null     datetime64[ns]
 2   gender      19 non-null     category      
dtypes: category(1), datetime64[ns](1), string(1)
memory usage: 2.0 KB


#### 2.2 'df_consultations'

In [474]:
# Analyse du DF 'df_consultations'

pd.set_option("display.max_seq_items", None)

# Exploration rapide des données

print("\n=== Informations générales ===")
display(df_consultations.info())

print("\n=== Statistiques descriptives (toutes colonnes) ===")
display(df_consultations.describe(include="all"))

print("\n=== Valeurs uniques par colonne (hors NaN) ===")
for col in df_consultations.columns:
    print(f"\n--- {col} ---")
    display(df_consultations[col].dropna().unique())

print("\n=== Nombre de valeurs nulles par colonne ===")
display(df_consultations.isna().sum())

print("=== Nombre de doublons par colonne ===")

for col in df_consultations.columns:
    nb_doublons = df_consultations[col].duplicated().sum()
    print(f"{col} : {nb_doublons}")


=== Informations générales ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   consultation_id    106 non-null    object
 1   patient_id         205 non-null    object
 2   date_consultation  120 non-null    object
 3   diagnostic         165 non-null    object
dtypes: object(4)
memory usage: 6.5+ KB


None


=== Statistiques descriptives (toutes colonnes) ===


,consultation_id,patient_id,date_consultation,diagnostic
count,106,205,120,165
unique,58,149,42,4
top,,p_032,N/ A,débuté
freq,48,5,43,46



=== Valeurs uniques par colonne (hors NaN) ===

--- consultation_id ---


...


--- diagnostic ---


array(['débuté', 'à refaire', 'nnull', 'terminé'], dtype=object)


=== Nombre de valeurs nulles par colonne ===


consultation_id      99
patient_id            0
date_consultation    85
diagnostic           40
dtype: int64

=== Nombre de doublons par colonne ===
consultation_id : 146
patient_id : 56
date_consultation : 162
diagnostic : 200


In [475]:
# La colonne 'consultation_id' contient 106 valeurs non nulles, 99 valeurs nulles, 58 valeurs uniques et une valeur non signifiante qui est : ' '.
# La colonne 'patient_id' contient 205 valeurs non nulles, 0 valeurs nulles, 149 valeurs uniques et aucune valeur non signifiante.
# La colonne 'date_consultation' contient 120 valeurs non nulles, 85 valeurs nulles, 42 valeurs uniques et 2 valeurs non signifiantes qui sont : 'not_a_date' et 'N/ A'.
# la colonne 'diagnostic' contient 165 valeurs non nulles, 40 valeurs nulles, 4 valeurs uniques et une valeur non signifiante : 'nnull'.

In [476]:
# vérification
n = len(df_consultations)

check = {}
for col in df_consultations.columns:
    dup = df_consultations[col].duplicated().sum()
    nun = df_consultations[col].nunique(dropna=False)
    check[col] = {"duplicates": dup, "nunique_including_NaN": nun, "n - nunique": n - nun}

import pandas as pd
display(pd.DataFrame(check).T)

,duplicates,nunique_including_NaN,n - nunique
consultation_id,146,59,146
patient_id,56,149,56
date_consultation,162,43,162
diagnostic,200,5,200


In [477]:
duplicated_patient_ids = (
    df_consultations["patient_id"]
    .value_counts()
    .loc[lambda x: x > 1]
    .head(5)
    .index
)


In [478]:
display(
    df_consultations[
        df_consultations["patient_id"].isin(duplicated_patient_ids)
    ]
)

,consultation_id,patient_id,date_consultation,diagnostic
11,NaN,p_037,NaN,débuté
27,,p_018,N/ A,NaN
38,NaN,p_037,NaN,débuté
45,,p_047,15/02/2022,à refaire
48,NaN,p_019,10/05/2022,débuté
60,c_183,p_037,N/ A,nnull
63,c_180,p_047,NaN,à refaire
92,c_153,p_032,NaN,nnull
94,,p_018,NaN,nnull
96,,p_047,31/10/2021,à refaire


In [479]:
import numpy as np

# Nettoyage des valeurs non-signifiantes

df_consultations["consultation_id"] = (
    df_consultations["consultation_id"]
    .str.strip()         # supprime les espaces
    .replace("", np.nan) # remplace les chaînes vides par NaN
)

df_consultations["date_consultation"] = df_consultations["date_consultation"].replace(
    ["not_a_date", "N/ A"], np.nan
)

df_consultations["diagnostic"] = df_consultations["diagnostic"].replace("nnull", np.nan)

In [480]:
# Vérification après nettoyage

print("\n=== Valeurs uniques par colonne (hors NaN) ===")
for col in df_consultations.columns:
    print(f"\n--- {col} ---")
    display(df_consultations[col].dropna().unique())

print("\n=== Nombre de valeurs nulles par colonne ===")
display(df_consultations.isna().sum())


=== Valeurs uniques par colonne (hors NaN) ===

--- consultation_id ---


array(['c_054', 'c_198', 'c_063', 'c_034', 'c_067', 'c_130', 'c_120',
       'c_124', 'c_142', 'c_045', 'c_135', 'c_161', 'c_077', 'c_194',
       'c_131', 'c_172', 'c_183', 'c_180', 'c_097', 'c_162', 'c_014',
       'c_013', 'c_081', 'c_017', 'c_043', 'c_153', 'c_000', 'c_117',
       'c_105', 'c_091', 'c_181', 'c_118', 'c_138', 'c_188', 'c_003',
       'c_088', 'c_190', 'c_064', 'c_004', 'c_149', 'c_141', 'c_041',
       'c_185', 'c_068', 'c_051', 'c_127', 'c_146', 'c_099', 'c_111',
       'c_061', 'c_182', 'c_036', 'c_150', 'c_095', 'c_100', 'c_020',
       'c_057'], dtype=object)


--- patient_id ---


array(['x_003', 'x_019', 'x_046', 'x_077', 'x_018', 'p_040', 'p_099',
       'x_069', 'p_097', 'p_060', 'p_075', 'p_037', 'x_038', 'p_036',
       'p_000', 'p_089', 'p_007', 'p_023', 'x_036', 'p_030', 'p_055',
       'p_053', 'p_071', 'x_029', 'x_032', 'x_042', 'x_027', 'p_018',
       'x_066', 'x_005', 'p_081', 'x_004', 'p_038', 'p_031', 'p_069',
       'x_013', 'x_064', 'p_011', 'p_079', 'p_001', 'x_008', 'x_063',
       'x_040', 'p_047', 'p_002', 'x_079', 'p_019', 'x_017', 'x_039',
       'x_024', 'x_048', 'x_022', 'p_050', 'x_041', 'x_071', 'p_065',
       'p_085', 'x_044', 'x_037', 'x_028', 'p_059', 'x_010', 'p_006',
       'x_053', 'p_068', 'x_074', 'p_098', 'x_045', 'p_035', 'x_051',
       'p_074', 'x_067', 'p_092', 'p_026', 'p_088', 'x_065', 'p_066',
       'x_026', 'x_014', 'p_083', 'p_032', 'p_010', 'p_034', 'x_052',
       'p_067', 'p_024', 'x_055', 'x_034', 'x_006', 'p_041', 'p_021',
       'p_015', 'x_061', 'p_022', 'x_009', 'p_070', 'x_001', 'x_025',
       'p_017', 'p_0


--- date_consultation ---


array(['29/02/2020', '25/02/2020', '27/08/2022', '16/04/2020',
       '03/06/2023', '13/01/2020', '12/07/2022', '17/05/2021',
       '15/02/2022', '10/05/2022', '22/11/2021', '14/06/2020',
       '26/06/2023', '22/08/2020', '02/11/2022', '21/09/2022',
       '30/09/2023', '03/09/2022', '31/10/2021', '08/11/2021',
       '28/02/2020', '27/05/2022', '25/03/2023', '10/07/2021',
       '26/07/2021', '05/03/2023', '17/09/2022', '09/05/2021',
       '19/10/2023', '08/04/2022', '22/10/2023', '15/11/2023',
       '03/05/2021', '09/01/2023', '25/05/2023', '09/10/2022',
       '11/05/2020', '06/06/2022', '30/09/2021', '14/05/2023'],
      dtype=object)


--- diagnostic ---


array(['débuté', 'à refaire', 'terminé'], dtype=object)


=== Nombre de valeurs nulles par colonne ===


consultation_id      147
patient_id             0
date_consultation    163
diagnostic            83
dtype: int64

In [481]:
# Typage explicite des colonnes

# Identifiant de consultation : chaîne de caractères
df_consultations["consultation_id"] = df_consultations["consultation_id"].astype("string")
# Identifiant patient : chaîne de caractères (clé de jointure)
df_consultations["patient_id"] = df_consultations["patient_id"].astype("string")
# Date de consultation : datetime
df_consultations["date_consultation"] = pd.to_datetime(
   df_consultations["date_consultation"],
    format="%d/%m/%Y"
)
# Diagnostic : variable catégorielle
df_consultations["diagnostic"] = df_consultations["diagnostic"].astype("category")


In [482]:
df_consultations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   consultation_id    58 non-null     string        
 1   patient_id         205 non-null    string        
 2   date_consultation  42 non-null     datetime64[ns]
 3   diagnostic         122 non-null    category      
dtypes: category(1), datetime64[ns](1), string(2)
memory usage: 5.3 KB


### 3. Jointure & Analyse

In [483]:
df_consultations.head()

,consultation_id,patient_id,date_consultation,diagnostic
0,<NA>,x_003,NaT,NaN
1,<NA>,x_019,2020-02-29,débuté
2,c_054,x_046,NaT,à refaire
3,c_198,x_077,NaT,débuté
4,<NA>,x_018,NaT,NaN


In [484]:
df_patients.head()

,patient_id,birth_date,gender
0,p_094,NaT,NaN
1,<NA>,NaT,NaN
2,<NA>,1977-08-20,NaN
3,<NA>,1942-09-03,F
4,<NA>,NaT,NaN


In [485]:
# Jointure consultations ↔ patients
df_joined = df_consultations.merge(
    df_patients[["patient_id"]],
    on="patient_id",
    how="left",
    indicator=True,
)

In [486]:
# Indicateur de validité
df_joined["patient_valide"] = df_joined["_merge"] == "both"

In [487]:
# Extraire le mois de consultations
df_joined["mois_consultation"] = (
    df_joined["date_consultation"].dt.to_period("M").astype(str)
)

In [488]:
# Calculer la proportion par mois  
resultat = (
    df_joined
    .groupby("mois_consultation")["patient_valide"]
    .mean()
    .reset_index(name="proportion_patient_id_valide")
)

In [489]:
resultat = resultat.sort_values("mois_consultation")
display(resultat)

,mois_consultation,proportion_patient_id_valide
0,2020-01,1.000000
1,2020-02,0.000000
2,2020-04,0.000000
3,2020-05,0.000000
4,2020-06,0.000000
5,2020-08,0.000000
6,2021-05,0.333333
7,2021-07,0.500000
8,2021-09,0.000000
9,2021-10,1.000000


### 4. Sauvegarde

In [490]:
outputs_dir.mkdir(exist_ok=True)

df_patients.to_parquet(outputs_dir / "patients.parquet", index=False)
df_consultations.to_parquet(outputs_dir / "consultations.parquet", index=False)
resultat.to_parquet(outputs_dir / "resultat_proportion.parquet", index=False)

In [491]:
df = pd.read_parquet(outputs_dir / "patients.parquet")
df.head()

,patient_id,birth_date,gender
0,p_094,NaT,NaN
1,<NA>,NaT,NaN
2,<NA>,1977-08-20,NaN
3,<NA>,1942-09-03,F
4,<NA>,NaT,NaN


In [492]:
df = pd.read_parquet(outputs_dir / "consultations.parquet")
df.head()

,consultation_id,patient_id,date_consultation,diagnostic
0,<NA>,x_003,NaT,NaN
1,<NA>,x_019,2020-02-29,débuté
2,c_054,x_046,NaT,à refaire
3,c_198,x_077,NaT,débuté
4,<NA>,x_018,NaT,NaN


In [493]:
df = pd.read_parquet(outputs_dir / "resultat_proportion.parquet")
df

,mois_consultation,proportion_patient_id_valide
0,2020-01,1.000000
1,2020-02,0.000000
2,2020-04,0.000000
3,2020-05,0.000000
4,2020-06,0.000000
5,2020-08,0.000000
6,2021-05,0.333333
7,2021-07,0.500000
8,2021-09,0.000000
9,2021-10,1.000000
